# Projeto Integrado CDIA5 - Grupo 3

# Sistema de Arquivo Distribuídos - HDFS e PySpark

# Coletando dados de 20 sites

## Visão Geral
Esse código pretende coletar dados de 20 sites dinâmicos diferentes e transformá-los em arquivos txt.

In [1]:
#Importações
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
#Criando uma nova pasta para salvar os sites
try:
    os.makedirs("sites")
except:
    pass

Existem sites que precisam de uma atenção especial em suas coletas, para isso é necessária a criação de 3 funções, as quais serão usadas como parâmetros futuramente.

In [ ]:
#Função para excluir classes "divs"
def get_text_exclude_divs(container):
    elements = container.find_elements(By.XPATH, "./*")
    elements = [elem.text for elem in elements if elem.tag_name != "div"]
    return "\n".join(elements)

#Função exclusiva para o site Versatille (Exclui classes "div" e "ul")
def get_text_versatille(container):
    container = container.find_element(By.TAG_NAME, "div")
    elements = container.find_elements(By.XPATH, "./*")
    elements = [elem.text for elem in elements if elem.tag_name != "div" and elem.tag_name != "ul"]
    return "\n".join(elements).replace("LEIA TAMBÉM:", "")

#Função exclusiva para o site stealthelook (exlcui classes "p" e "h2", exclui tambem tudo abaixo do "Leia mais"(Não relevante para o trabalho))
def get_text_stealthelook(container):
    elements = container.find_elements(By.XPATH, "./*")
    elements = [elem.text for elem in elements if elem.tag_name == "p" or elem.tag_name == "h2"]
    return "\n".join(elements).split("LEIA MAIS")[0]

15 site podem ser feitos de maneira simples, fornecendo a url e a Classe, Id ou Xpath que será raspada.

5 sites recebem a url e a Classe ou Xpath que contem o texto principal e uma das funções feitas acima.

In [ ]:
#Urls que serão usados para análise 
urls = [
    ("https://www.tudosobremake.com.br/noticia/mascara-de-argila-no-rosto-saiba-como-usar-o-produto-antes-e-depois-de-fazer-uma-maquiagem-de-festa_a20067/1", By.CLASS_NAME, "article-content__text-block", None),
    ("https://www.loreal-paris.com.br/argila-para-o-rosto-saiba-como-inserir-a-mascara-facial-na-sua-rotina-de-cuidados-com-a-pele#:~:text=As%20m%C3%A1scaras%20de%20argila%20devem,e%20antes%20do%20creme%20hidratante.",By.CLASS_NAME, "rich-text", None),
    ("https://terralnatural.com.br/blogs/news/mascara-de-argila", By.CLASS_NAME, "article__content", None),
    ("https://www.dermaclub.com.br/blog/todos-os-temas/argila-verde-branca-preta-como-usar-na-pele-quais-sao-os-tipos-para-que-serve-conheca-os-beneficios_a9283.html", By.CLASS_NAME, "l-blog-article__content", None),
    ("https://quintaldermocosmeticos.com.br/blogs/quintal/argila-beleza-que-vem-do-solo?psafe_param=1&gad=1&gclid=CjwKCAjwvJyjBhApEiwAWz2nLSHEs0mQehb5YPbETSbRzH8-DKBCcLmc0OplQfX_8UOgHQm1sQsPuhoCZJ8QAvD_BwE", By.CLASS_NAME, "rte", None),
    ("https://odia.ig.com.br/d-mulher/2017-01-26/argilas-para-o-rosto-fazem-drenagem-e-desintoxicam-a-pele.html", By.CLASS_NAME, "article-body", None),
    ("https://www.fashionbubbles.com/beleza/mascara-de-argila/", By.CLASS_NAME, "content-wrap", None),
    ("https://manipurafarmacia.com.br/mascaras-de-argila-tudo-sobre-elas/", By.CLASS_NAME, "article-inner", None),
    ("https://revistaquem.globo.com/QUEM-Inspira/noticia/2019/05/receita-caseira-aprenda-fazer-mascara-facial-detox-da-blogueira-mari-saad.html", By.CLASS_NAME, "protected-content", None),
    ("https://simpleorganic.com.br/blogs/simple-blog/argila-para-os-rosto-como-usar-e-diluir", By.CLASS_NAME, "Article__Content", None),
    ("https://carenb.com/blogs/beauty-journal/mascara-de-argila-saiba-como-utilizar-e-seus-beneficios", By.CLASS_NAME, "article-template", None),
    ("https://www.ecycle.com.br/limpeza-de-pele-com-argila/", By.ID, "entry-content", None),
    ("https://hidrabene.com.br/post/o-que-a-mascara-de-argila-faz-no-rosto/", By.XPATH, "//div[@class='conteudo has-text-justified']", None),    
    ("https://tropicabotanica.com.br/quais-o-tipos-de-argila-indicados-para-cada-rosto/", By.CLASS_NAME, "entry-content", None),
    ("https://www.natura.com.br/blog/pele/mascara-de-argila-conheca-os-beneficios-e-o-jeito-certo-de-aplicar", By.XPATH, '//*[@id="root"]/div[3]/div/article/div[2]/div[2]', None),
    #Sites que precisam de mais atenção    
    ("https://cabelo.com.br/argila-no-cabelo-veja-os-beneficios-de-usar-o-produto-no-cabelo/", By.CLASS_NAME, "entry-content", get_text_exclude_divs),    
    ("https://pontodamulher.com.br/mascara-de-argila-verde", By.CLASS_NAME, "col-lg-9", get_text_exclude_divs),
    ("https://www.belezanaweb.com.br/loucas-por-beleza/como-usar-argila-no-cabelo/", By.CLASS_NAME, "post-content", get_text_exclude_divs),
    ("https://versatille.com/skin-care-3-receitas-caseiras-e-faceis-com-argila-e-oleos-essenciais/", By.CLASS_NAME, "edgtf-post-text", get_text_versatille),
    ("https://stealthelook.com.br/argila-no-cabelo-quais-sao-os-beneficios-e-como-usar-cada-uma-delas/", By.XPATH, "//main/article", get_text_stealthelook), 
]

Nessa celula está sendo realizadas as operações de automação usando a biblioteca Selenium WebDriver.
Usaremos o Chrome como navegador.

In [ ]:
#Operação automatizada usando Selenium WebDriver
driver = webdriver.Chrome() #Abre um Chrome vazio
for index, url in enumerate(urls): #Enumera as urls
    driver.get(url[0]) #Começa o processo apartir da url 0
    time.sleep(3)
    if url[3] is not None: #Para as urls que necessitam funções
        text = url[3](driver.find_element(url[1], url[2]))
    else: #Se a url não usar função simplesmente extrai o texto usando o seletor url[1] e ur[2]
        text = driver.find_element(url[1], url[2]).text
    with open(f"sites/site{index+1}_content.txt", "w", encoding="utf-8") as f:
        f.write(text)#salva o texto extraído
    time.sleep(3)

driver.quit() #Fecha o navegador e encerra a instância do WebDriver.

Cada site visitado será salvo como arquivos txt's separadamente no diretório onde o arquivo está salvo. 
A aplicação pode levar um longo tempo para ser finalizada.

# Coletando dados do Facebook

Para podermos usar o Facebook é necessário ter uma conta, caso contrario conseguimos visualizar apenas uma publicação de cada vez, para resolver esse problema uma conta foi criada para podermos acessar os dados sem mais problemas, essa conta não pertence a ninguém especifico e foi criada apenas para fins da realização desse trabalho.
A url fornecida nos levara diretamente para a área destina a publicações relacionas a máscaras de argila.

In [2]:
fb_url = "https://www.facebook.com/search/posts/?q=mascara%20de%20argila&__tsid__=0.5023192808728404&__epa__=SERP_TAB&__eps__=SERP_POSTS_TAB"

user = "yadoji2440@mevori.com"
pw = "mevori.com@yadoji2440"

In [5]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get(fb_url)
time.sleep(2)
elem = driver.switch_to.active_element
elem.send_keys(user)
elem.send_keys(Keys.TAB)
time.sleep(3)
elem = driver.switch_to.active_element
elem.send_keys(pw)
elem.send_keys(Keys.ENTER)
time.sleep(5)
driver.get(fb_url)
time.sleep(3)

posts = []
old_posts_len = -1
i = 1
while len(posts) > old_posts_len:
    attr = 'data-ad-preview="message"'
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    old_posts_len = len(posts)
    posts = driver.find_elements(By.XPATH, f'//div[@{attr}]')
    time.sleep(1)
    novos_posts = len(posts) - old_posts_len
    #print(f"encontrados {novos_posts} novos posts")
    for post in posts[old_posts_len::]:
        attr = 'role="button"'
        for button in post.find_elements(By.XPATH, f'//div[@{attr}]'):
            if button.text.strip() == "Ver mais":
                #print("clicando no ver mais")
                driver.execute_script("arguments[0].click();", button)
                break
        time.sleep(2)
        with open(f"fb_post_{i+1}.txt", "w", encoding="utf-8") as f:
            f.write(post.text)
        #print(post.text)
        #print(f"finalizado post {i}")
        i += 1
print(f"encontrados {len(posts)} posts")

C:\Users\positivo\AppData\Local\Temp/ipykernel_3392/3371692306.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


encontrados 82 posts


A aplicação pode demorar para finalizar, não deixe o computador desligar

# Juntando os arquivos txt

Antes de realizarmos todo o processo PySpark para contagem de palavras precisamos juntar todos os arquivos txt coletados do facebook em um único, para isso usaremos o comando abaixo que juntará todos os arquivos txt's localizados na pasta que o caderno está.

Se pode escolher alguma coisa ou se tem que abritario, por exemplo, se o texto for cansativo?, se o texto tiver ordem nas palavras, por exemlo, argila é a primeira no facebook, e primeira nos site então isso vale sei la 100 pontos

In [ ]:
copy fb*.txt merged_fb.txt

# Contando palavras com o PySpark

In [7]:
#Instalando a biblioteca
#!pip install pyspark py4j
#!pip install unidecode
#Importações
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import explode, col
from pyspark.sql.functions import lower
from pyspark.sql.functions import regexp_extract
from pyspark.ml.feature import StopWordsRemover
from unidecode import unidecode

In [3]:
#Iniciando uma sessão no Spark
spark = SparkSession \
          .builder \
          .appName("Máscara_de_argila_Grupo3") \
          .getOrCreate()


spark

In [4]:
#Função limpa, ordena e conta palavras
def get_count(filename, lang="portuguese"):
    with open(filename, encoding="utf-8") as f:
        merged = f.read()
    with open(filename, "w") as f:
        f.write(unidecode(merged))
    site = spark.read.text(filename)
    lines = site.select(
        split(site.value, " ").alias("line")
    )
    words = lines.select(
        explode(col("line")).alias("word")
    )
    lowered_words = words.select(
        lower(
            col("word")
        ).alias("lowered_word")
    )
    cleaned_words = lowered_words.select(
        regexp_extract(
            col("lowered_word"), # Indico a coluna
            "[a-z]+", # Seleciona somente o que parece uma palavra
            0 # 0 significa dar o match em toda expressão regular
        ).alias("cleaned_word")
    )
    words_nonull = cleaned_words.where(
        col("cleaned_word") != ""
    )
    words_nonull = words_nonull.select(
        col("cleaned_word").alias("word")
    )
    words_lst_df = words_nonull.select(
        split(
            col("word"),
            " "
        ).alias("word")
    )
    swr = StopWordsRemover(
        inputCol="word",
        outputCol="meaningful",
        stopWords=StopWordsRemover.loadDefaultStopWords(lang),
    )
    swr_df = swr.transform(words_lst_df)

    meaningful = swr_df.select(
        (col("meaningful")[0]).alias("meaningful")
    )
    meaningful_nonull = meaningful.where(
        col("meaningful") != "null"
    )
    words_counts = meaningful_nonull.groupby(
        col("meaningful")
    ).count()

    return words_counts.orderBy(
        col("count"), 
        ascending=False
    )

In [5]:
#Mostrando o resultado do facebook
fb_merged = get_count("merged_fb.txt")
fb_merged.show()

+----------+-----+
|meaningful|count|
+----------+-----+
|      pele|  264|
|    argila|  262|
|   mascara|  186|
|       nao|  106|
|    facial|   92|
|     rosto|   61|
|      agua|   61|
|         r|   60|
|      oleo|   60|
|    rotina|   53|
|       pra|   52|
|  produtos|   51|
|hidratante|   51|
|      voce|   46|
|      rosa|   45|
|       uso|   44|
|        ja|   43|
|    semana|   42|
|oleosidade|   41|
|     verde|   41|
+----------+-----+
only showing top 20 rows



In [8]:
#Mostrando o resultado dos sites
artigos = {}
for site in os.listdir("sites"):
    #print(site)
    artigos[site] = get_count(f"sites/{site}")
    # se atual for melhor que o best_article, substituir
artigos["site10_content.txt"].show()

+-------------+-----+
|   meaningful|count|
+-------------+-----+
|       argila|   32|
|         pele|   24|
|      mascara|   21|
|        peles|    8|
|         alem|    6|
|        conta|    5|
|         rica|    5|
|      remover|    4|
|         voce|    4|
|       diluir|    4|
|      argilas|    4|
|        rosto|    4|
|          sao|    4|
|       outras|    3|
|        assim|    3|
| desidratadas|    3|
|mineralizante|    3|
|        ideal|    3|
|      aplicar|    3|
|        disso|    3|
+-------------+-----+
only showing top 20 rows



In [ ]:
# for artigo in artigos:
#     comparar(artigos[artigo], fb_merged)